## Table of Contents
1. [Setup and Imports](#setup)
2. [Discover Centrality Artifacts](#discover)
3. [Load and Inspect Centrality Data](#load)
4. [Top-K Rankings by Metric](#top-k)
5. [Centrality Distributions](#distributions)
6. [Connector vs Hub Analysis](#connector-hub)
7. [Interpretation](#interpretation)
8. [Write Report Outputs](#write-outputs)
9. [Reproducibility Notes](#reproducibility)

In [ ]:
# ============================================================================
# SETUP AND IMPORTS
# ============================================================================

import json
from pathlib import Path
from datetime import datetime
import warnings

import pandas as pd
import polars as pl
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Project paths
REPO_ROOT = Path.cwd().parent.parent
RESULTS_DIR = REPO_ROOT / "results"
ANALYSIS_DIR = RESULTS_DIR / "analysis"
TABLES_REPORT_DIR = RESULTS_DIR / "tables" / "report"
FIGURES_REPORT_DIR = RESULTS_DIR / "figures" / "report"
WARNINGS_LOG = TABLES_REPORT_DIR / "_warnings.log"

# Notebook identity
NOTEBOOK_ID = "nb03"
NOTEBOOK_NAME = "centrality__rankings_and_mechanisms"

# Plotting settings
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("husl")

# Ensure output directories exist
TABLES_REPORT_DIR.mkdir(parents=True, exist_ok=True)
FIGURES_REPORT_DIR.mkdir(parents=True, exist_ok=True)

print(f"Analysis dir exists: {ANALYSIS_DIR.exists()}")

In [ ]:
# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def append_warning(message: str, notebook_id: str = NOTEBOOK_ID):
    """Append a warning to the consolidated warnings log."""
    timestamp = datetime.now().isoformat()
    with open(WARNINGS_LOG, "a") as f:
        f.write(f"[{timestamp}] [{notebook_id}] {message}\n")
    print(f"WARNING: {message}")

def safe_load_parquet(path: Path) -> pl.DataFrame | None:
    """Safely load a parquet file, returning None if it fails."""
    try:
        return pl.read_parquet(path)
    except Exception as e:
        append_warning(f"Failed to load {path.name}: {e}")
        return None

def get_top_k(df: pl.DataFrame, metric_col: str, id_col: str, k: int = 20) -> pd.DataFrame:
    """Get top-k entries by a metric with stable tie-breaking."""
    return (
        df
        .sort([metric_col, id_col], descending=[True, False])  # Tie-break by ID ascending
        .head(k)
        .with_row_index("rank", offset=1)
        .to_pandas()
    )

<a id="discover"></a>
## 2. Discover Centrality Artifacts

Search for files containing centrality-related keywords.

In [ ]:
# ============================================================================
# DISCOVER CENTRALITY ARTIFACTS
# ============================================================================

centrality_keywords = ["centrality", "pagerank", "betweenness", "closeness", "eigen", "degree"]

# Search in analysis directory
analysis_files = list(ANALYSIS_DIR.glob("*.parquet")) + list(ANALYSIS_DIR.glob("*.csv"))
centrality_candidates = [
    f for f in analysis_files 
    if any(kw in f.name.lower() for kw in centrality_keywords)
]

print(f"Found {len(centrality_candidates)} centrality-related artifacts:")
for cf in centrality_candidates:
    print(f"  - {cf.name}")

# Select primary centrality file
primary_centrality_file = None
for candidate in ["airport_centrality.parquet", "centrality.parquet"]:
    path = ANALYSIS_DIR / candidate
    if path.exists():
        primary_centrality_file = path
        break

if primary_centrality_file:
    print(f"\n✅ Primary centrality file: {primary_centrality_file.name}")
else:
    append_warning("No primary centrality file found")

<a id="load"></a>
## 3. Load and Inspect Centrality Data

In [ ]:
# ============================================================================
# LOAD AND INSPECT CENTRALITY DATA
# ============================================================================

centrality_df = None
if primary_centrality_file:
    centrality_df = safe_load_parquet(primary_centrality_file)

if centrality_df is not None:
    print(f"Centrality data shape: {centrality_df.shape}")
    print(f"Columns: {centrality_df.columns}")
    display(centrality_df.head(10).to_pandas())
    
    # Identify node ID column
    id_col = next((c for c in ["airport", "node", "ORIGIN", "id", "name"] if c in centrality_df.columns), None)
    print(f"\nNode ID column: {id_col}")
    
    # Identify numeric metric columns
    numeric_cols = [c for c in centrality_df.columns if centrality_df[c].dtype in [pl.Float64, pl.Float32, pl.Int64, pl.Int32]]
    metric_cols = [c for c in numeric_cols if c != id_col and not c.startswith("_")]
    print(f"Metric columns: {metric_cols}")
else:
    print("Not available: centrality data could not be loaded")
    id_col = None
    metric_cols = []

<a id="top-k"></a>
## 4. Top-K Rankings by Metric

Compute top-20 rankings for each centrality metric with stable tie-breaking.

In [ ]:
# ============================================================================
# TOP-K RANKINGS BY METRIC
# ============================================================================

top_k_results = {}
K = 20

if centrality_df is not None and id_col and len(metric_cols) > 0:
    for metric in metric_cols:
        top_k = get_top_k(centrality_df, metric, id_col, K)
        top_k_results[metric] = top_k
        
        print(f"\n{'='*60}")
        print(f"TOP {K} BY {metric.upper()}")
        print(f"{'='*60}")
        display(top_k[["rank", id_col, metric]].head(10))
else:
    print("Not available: cannot compute top-k rankings")

In [ ]:
# ============================================================================
# PLOT TOP-K BAR CHARTS
# ============================================================================

if len(top_k_results) > 0:
    for metric, top_k in top_k_results.items():
        fig, ax = plt.subplots(figsize=(12, 8))
        
        colors = sns.color_palette("viridis", len(top_k))
        bars = ax.barh(range(len(top_k)), top_k[metric], color=colors)
        ax.set_yticks(range(len(top_k)))
        ax.set_yticklabels(top_k[id_col])
        ax.invert_yaxis()
        ax.set_xlabel(metric.replace("_", " ").title())
        ax.set_title(f"Top {K} Airports by {metric.replace('_', ' ').title()}")
        
        plt.tight_layout()
        fig_path = FIGURES_REPORT_DIR / f"{NOTEBOOK_ID}_centrality_top20__{metric}.png"
        plt.savefig(fig_path, dpi=150)
        plt.show()
        print(f"✅ Saved: {fig_path.name}")

<a id="distributions"></a>
## 5. Centrality Distributions

Visualize the distribution of centrality metrics to identify concentration patterns.

In [ ]:
# ============================================================================
# CENTRALITY DISTRIBUTIONS
# ============================================================================

if centrality_df is not None and len(metric_cols) > 0:
    n_metrics = len(metric_cols)
    n_cols = min(3, n_metrics)
    n_rows = (n_metrics + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(5*n_cols, 4*n_rows))
    axes = np.atleast_2d(axes).flatten()
    
    for i, metric in enumerate(metric_cols):
        ax = axes[i]
        values = centrality_df[metric].to_numpy()
        
        # Use log scale if values span multiple orders of magnitude
        use_log = (values.max() / (values.min() + 1e-10)) > 100
        
        ax.hist(values, bins=50, edgecolor="white", alpha=0.8, log=use_log)
        ax.set_xlabel(metric.replace("_", " ").title())
        ax.set_ylabel("Frequency" + (" (log)" if use_log else ""))
        ax.set_title(f"{metric} Distribution")
        
        # Add mean line
        ax.axvline(values.mean(), color="red", linestyle="--", alpha=0.7, label=f"Mean: {values.mean():.2e}")
        ax.legend(fontsize=8)
    
    # Hide unused axes
    for j in range(i+1, len(axes)):
        axes[j].set_visible(False)
    
    plt.tight_layout()
    fig_path = FIGURES_REPORT_DIR / f"{NOTEBOOK_ID}_centrality_distributions.png"
    plt.savefig(fig_path, dpi=150)
    plt.show()
    print(f"✅ Saved: {fig_path.name}")
else:
    print("Not available: cannot plot distributions")

<a id="connector-hub"></a>
## 6. Connector vs Hub Analysis

Identify airports that are high-betweenness but not top-degree (connectors/bridges).

In [ ]:
# ============================================================================
# CONNECTOR VS HUB ANALYSIS
# ============================================================================

if centrality_df is not None:
    # Look for betweenness and degree columns
    betweenness_col = next((c for c in metric_cols if "betweenness" in c.lower()), None)
    degree_col = next((c for c in metric_cols if "degree" in c.lower() or "strength" in c.lower()), None)
    
    if betweenness_col and degree_col:
        print(f"Comparing {betweenness_col} vs {degree_col}")
        
        # Compute percentile ranks
        df_analysis = centrality_df.to_pandas()
        df_analysis["betweenness_pctile"] = df_analysis[betweenness_col].rank(pct=True)
        df_analysis["degree_pctile"] = df_analysis[degree_col].rank(pct=True)
        
        # Identify connectors: high betweenness (>90th) but lower degree (<75th)
        connectors = df_analysis[
            (df_analysis["betweenness_pctile"] > 0.90) & 
            (df_analysis["degree_pctile"] < 0.75)
        ].sort_values(betweenness_col, ascending=False)
        
        print(f"\n🔗 POTENTIAL CONNECTORS (high betweenness, moderate degree): {len(connectors)}")
        if len(connectors) > 0:
            display(connectors[[id_col, betweenness_col, degree_col, "betweenness_pctile", "degree_pctile"]].head(10))
        
        # Identify mega-hubs: high on both
        mega_hubs = df_analysis[
            (df_analysis["betweenness_pctile"] > 0.95) & 
            (df_analysis["degree_pctile"] > 0.95)
        ].sort_values(degree_col, ascending=False)
        
        print(f"\n🏢 MEGA-HUBS (high on both): {len(mega_hubs)}")
        if len(mega_hubs) > 0:
            display(mega_hubs[[id_col, betweenness_col, degree_col]].head(10))
    else:
        print(f"Not available: could not find betweenness ({betweenness_col}) or degree ({degree_col}) columns")
else:
    print("Not available: centrality data not loaded")

<a id="interpretation"></a>
## 7. Interpretation

### Key Findings (Evidence-Grounded)

*(Populated after running cells above)*

### Mechanistic Explanation

- **Betweenness centrality**: Measures bridging/brokerage roles - airports on many shortest paths
- **PageRank/Eigenvector**: Measures global influence - connected to well-connected airports
- **Degree/Strength**: Measures local connectivity - number/volume of direct connections

### Alternative Explanations
1. Geographic positioning may drive betweenness (central US locations)
2. Airline hub strategy decisions may drive degree concentration

### Evidence Links
- Table: `results/tables/report/nb03_centrality_top20_by_metric.csv`
- Figures: `results/figures/report/nb03_centrality_top20__*.png`
- Figure: `results/figures/report/nb03_centrality_distributions.png`

<a id="write-outputs"></a>
## 8. Write Report Outputs

In [ ]:
# ============================================================================
# WRITE REPORT OUTPUTS
# ============================================================================

# Combine all top-k results into a single table
if len(top_k_results) > 0:
    combined_rows = []
    for metric, top_k in top_k_results.items():
        for _, row in top_k.iterrows():
            combined_rows.append({
                "metric": metric,
                "rank": row["rank"],
                "airport": row[id_col],
                "value": row[metric]
            })
    
    combined_df = pd.DataFrame(combined_rows)
    combined_path = TABLES_REPORT_DIR / f"{NOTEBOOK_ID}_centrality_top20_by_metric.csv"
    combined_df.to_csv(combined_path, index=False)
    print(f"✅ Wrote: {combined_path}")

print(f"\n📋 All {NOTEBOOK_ID} outputs written.")

<a id="reproducibility"></a>
## 9. Reproducibility Notes

### Input Files Consumed
- `results/analysis/airport_centrality.parquet`

### Assumptions Made
1. Centrality metrics were computed on the directed weighted airport network
2. Betweenness is normalized by the number of node pairs
3. PageRank uses default damping factor (typically 0.85)

### Sorting/Ordering
- Tie-breaking: by airport code ascending when metrics are equal
- All rankings are stable and deterministic

### Caveats
- Centrality semantics depend on network construction choices (directed vs undirected, weighted vs unweighted)
- Results may differ if computed on largest connected component only

### Outputs Generated
| Artifact | Path |
|----------|------|
| Combined Rankings | `results/tables/report/nb03_centrality_top20_by_metric.csv` |
| Top-K Figures | `results/figures/report/nb03_centrality_top20__*.png` |
| Distributions | `results/figures/report/nb03_centrality_distributions.png` |